In [1]:
import folium
import pandas as pd

In [2]:
from folium.plugins import MarkerCluster, MousePosition
from folium.features import DivIcon

In [3]:
df= pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [4]:
launch_site_columns=['Launch Site','Lat', 'Long','class']
launch_sites_df = df[launch_site_columns]
launch_sites_df.head()

,Launch Site,Lat,Long,class
0,CCAFS LC-40,28.562302,-80.577356,0
1,CCAFS LC-40,28.562302,-80.577356,0
2,CCAFS LC-40,28.562302,-80.577356,0
3,CCAFS LC-40,28.562302,-80.577356,0
4,CCAFS LC-40,28.562302,-80.577356,0


In [5]:
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start= 10)
site_map


In [6]:
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))

#create a blue circle at NASA Johnson Space centers coordinate with icon showing name
marker = folium.map.Marker(
    nasa_coordinate,
    #create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color: #d35400;"><b>%s</b></div>' %'NASA JSC',
    )
)
site_map.add_child(circle)
site_map.add_child(marker)

In [7]:
launch_coordinates_df = df[['Launch Site','Lat', 'Long']].groupby('Launch Site').mean()
df_2 = launch_coordinates_df.reset_index()


In [8]:
site_map=folium.Map(location=nasa_coordinate, zoom_start=5)

for name, lat, long in zip(df_2['Launch Site'], df_2.Lat, df_2.Long):
    folium.Circle(
        [lat,long],
        radius=1000,
        color='#35400',
        fill=True
    ).add_child(folium.Popup(name)).add_to(site_map)
    
    folium.map.Marker(
        [lat,long],
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html = '<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % name,
        )
    ).add_to(site_map)
    


In [9]:
site_map

# Marking the successful/ failed launches for ecah site on the map

In [10]:
spacex_df = df[['Launch Site', 'Lat', 'Long', 'class']]
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [11]:
marker_cluster = MarkerCluster()
launch_sites_df['marker_color']= launch_sites_df['class'].apply(lambda x: 'green' if x == 1 else 'red')
launch_sites_df.isnull().sum()

C:\Users\Sherrif\AppData\Local\Temp\ipykernel_14036\499989433.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  launch_sites_df['marker_color']= launch_sites_df['class'].apply(lambda x: 'green' if x == 1 else 'red')


Launch Site     0
Lat             0
Long            0
class           0
marker_color    0
dtype: int64

In [12]:
spacex_df.head()
launch_sites_df.head()

,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red


In [13]:
# add marker cluster to current site map
site_map.add_child(marker_cluster)

for index, record in launch_sites_df.iterrows():
    site = record[0]
    lat = record[1]
    long = record [2]
    outcome = record [3]
    marker = folium.Marker(
        location=[lat,long],
        icon = folium.Icon(color='white', icon_color=launch_sites_df.iloc[index,4],
        popup=outcome,
    ))
    marker_cluster.add_child(marker)

site_map
    
    

# Calculating the distance between launch sites and its proximities

In [14]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [15]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [16]:
lat_ = 28.56374
long_ = -80.56809
launch_site_lat = 28.562302
launch_site_long = -80.577356

distance_coastline = calculate_distance(launch_site_lat, launch_site_long,lat_,long_)
distance_coastline

0.9192388878379587

In [17]:
distance_marker = folium.Marker(
    [lat_,long_],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html= '<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
    )
)

lines = folium.PolyLine(
    locations= [
        [lat_,long_],
        [launch_site_lat, launch_site_long]
    ],
    weight = 1,
    popup='distance to coastline'
)

site_map.add_child(lines)